In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import time
import json
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.Session()
import keras
import keras_metrics
import random

import import_ipynb
from reading_splitting_dataset_functions import *
from __future__ import print_function
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from sklearn.model_selection import StratifiedGroupKFold

In [95]:
### Hier AUSWAHL TREFFEN!

# Parameter für Kreuzvalidierung
n_splits = 2
n_epochs = 100
batch_size = 64


In [3]:
data_roi = open_js_file('data_preprocessed_roi.JSON')
print('length ROI:', len(data_roi))

length ROI: 33676


In [4]:
df_roi, fid_roi, v_roi, lva_roi, lha_roi = get_acceleration_fid_v_labels(data_roi)

In [5]:
print(df_roi.shape)
print(fid_roi.shape)
print(v_roi.shape)
print(lva_roi.shape)
print(lha_roi.shape)

(33676, 512, 3)
(33676,)
(33676,)
(33676,)
(33676,)


In [6]:
l_roi = labels_roi(lva_roi, lha_roi)
print(l_roi[1:10])

[2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [7]:
#amount splits into test and trainingsdata such every fid was exactly once in testdata
#with n_splits->
cv = StratifiedGroupKFold(n_splits)
splits =  cv.split(df_roi, l_roi, fid_roi)
splits = list(splits) # Generatoren sind schlecht wiederverwendbar. Deshalb Liste draus machen.

In [24]:
[*df_roi.shape[1:],1]

[512, 3, 1]

In [100]:
def create_conv_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(
            kernel_size = (12,3),
            filters = 10,
            strides = (3,1),
            input_shape = [*df_roi.shape[1:],1], # Das Sternchen sorgt für's "Entpacken". So können zwei Listen zusammengehängt werden.
            activation = "relu"
        ),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.MaxPool2D(
            pool_size = (5,1)
        ),
        tf.keras.layers.Conv2D(
            kernel_size = (8,1),
            filters = 10,
            activation = "relu"
        ),
        tf.keras.layers.MaxPool2D(
            pool_size = (7,1)
        ),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(
            units = 10,
            activation = "relu"
        ),
        tf.keras.layers.Dense(
            units = 4,
            activation = "softmax"
        )
    ])
    
    return model

In [101]:
def bring_in_right_shape_3D(x_train, y_train, x_test, y_test, num_classes=4, num_features=512*3):
    #from tensorflow import to_categorical
    #reshape y
    y_train = y_train.reshape((-1,1))
    y_test = y_test.reshape((-1,1))
    
    #cast to np.float32
    x_train = x_train.astype(np.float32)
    y_train = y_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_test = y_test.astype(np.float32)
    
    # Compute the categorical classes (doenst requested when binary classification)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)
    
    x_train = x_train.reshape(*x_train.shape, 1)
    x_test  = x_test.reshape([*x_test.shape, 1])
    
    return x_train, y_train, x_test, y_test


In [102]:
test_acc = []
train_acc = []
for train_idxs, test_idxs in splits:        
        x_train, y_train, x_test, y_test = bring_in_right_shape_3D(df_roi[train_idxs],
                                                                     l_roi[train_idxs],
                                                                     df_roi[test_idxs],
                                                                     l_roi[test_idxs])
        model = create_conv_model()
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        H = model.fit(x_train, y_train,
                      batch_size = batch_size,
                      validation_data = (x_test, y_test),
                      epochs = n_epochs)
        
        test_acc.append(H.history["val_accuracy"][-1])
        train_acc.append(H.history["accuracy"][-1])
        
print("Train Accuray = ", train_acc, "mit Mittelwert", np.mean(train_acc))
print("Test Accuray = ", test_acc, "mit Mittelwert", np.mean(test_acc))
        

Epoch 1/100
260/260 [==============================] - 13s 24ms/step - loss: 0.8788 - accuracy: 0.6116 - val_loss: 0.6437 - val_accuracy: 0.7729
Epoch 2/100
260/260 [==============================] - 5s 19ms/step - loss: 0.4791 - accuracy: 0.8149 - val_loss: 0.5065 - val_accuracy: 0.8270
Epoch 3/100
260/260 [==============================] - 5s 19ms/step - loss: 0.3857 - accuracy: 0.8516 - val_loss: 0.4200 - val_accuracy: 0.8556
Epoch 4/100
260/260 [==============================] - 5s 20ms/step - loss: 0.3311 - accuracy: 0.8749 - val_loss: 0.3912 - val_accuracy: 0.8537
Epoch 5/100
260/260 [==============================] - 5s 21ms/step - loss: 0.2950 - accuracy: 0.8854 - val_loss: 0.3418 - val_accuracy: 0.8835
Epoch 6/100
260/260 [==============================] - 5s 21ms/step - loss: 0.2654 - accuracy: 0.8966 - val_loss: 0.3392 - val_accuracy: 0.8719
Epoch 7/100
260/260 [==============================] - 5s 18ms/step - loss: 0.2481 - accuracy: 0.9030 - val_loss: 0.3122 - val_accuracy

In [103]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 167, 1, 10)        370       
_________________________________________________________________
dropout_8 (Dropout)          (None, 167, 1, 10)        0         
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 33, 1, 10)         0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 26, 1, 10)         810       
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 3, 1, 10)          0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 30)                0         
_________________________________________________________________
dense_70 (Dense)             (None, 10)              